### Get All Locations

In [ ]:
from flair.data import Sentence, Token
from flair.models import SequenceTagger

books = ['Outlander', 'Dragonfly in Amber', 'Voyager', 'Drums of Autumn', 'The Fiery Cross',
         'A Breath of Snow and Ashes',
         'An Echo in the Bone', 'Written in My Own Heart’s Blood']
extras = ['Other Books by this Author', 'About the Author']
bookstarts = [50, 17287, 37378, 61857, 89432, 119494, 152540, 177800, 202059]
f = open("data.txt", "r", encoding="utf8")
booknum = 3

book = ''
i = 0
for line in f:
    if i < bookstarts[booknum] and i > bookstarts[booknum - 1]:
        book = book + line
    i = i + 1
print(len(book))

# load tagger
tagger_ner = SequenceTagger.load("flair/ner-english-ontonotes-fast")
tagger_pos = SequenceTagger.load("flair/pos-english")


def ner_tag(text):
    sentence_ner = Sentence(text)

    # predict NER tags
    tagger_ner.predict(sentence_ner)
    return sentence_ner


def pos_tag(text):
    sentence_pos = Sentence(text.split(' '), use_tokenizer=True)

    # predict NER tags
    tagger_pos.predict(sentence_pos)
    return sentence_pos


def get_person_before(sentence_ner, token):
    if token.text == 'it':
        return token.text
    if token.text == 'I' or token.text == 'me':
        return 'Claire'
    prev_person = ''
    for entity in sentence_ner.get_spans('ner'):
        if entity.tag == 'PERSON' and entity.start_position < token.start_position:
            prev_person = entity.text
        elif entity.start_position > token.start_position:
            break
    return prev_person


import re


def split_into_chapters(booknum, text):
    if booknum < 7:
        return re.split(r'(\\n[0-9]+\\n\\n)', book)
    else:
        # needs refinement
        return re.split(r'[A-Z ]+\\n\\n', book)


def split_into_paragraphs(text):
    splitted = re.split('(\*\*\*)', text)
    return [x for x in splitted if x != '']


chapters = re.split(r'\n[0-9]+\n\n')

### From loc get lon and lat